In [1]:
from pandas import read_csv
from scripts.dataset_preprocessing import (
    binning_date_by_period,
    label_encode_dataframe,
    drop_outliers,
    drop_shared_outliers
)

In [2]:
dataframe = read_csv("../data/raw/marketing_campaign.csv")
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

### UNDEBATTABLE ACTIONS
As we've seen with the exploratory anaylsis, some feature are useless because of the unique values, or useless in a model.  
The **Dt_Customer** is interesting to be plot, however in a model the multiple categories are irelevant.  
The others categoricals values will be either LabelEncode or OneHotEncode, we coul LabelEncode it in a first place.  
***

In [3]:
dataframe.drop(columns=["ID", "Z_Revenue", "Z_CostContact"], inplace=True)
dataframe["Dt_Customer"] = dataframe["Dt_Customer"].apply(
    binning_date_by_period, args=("Year",)
)

In [4]:
categorical_columns = ["Education", "Marital_Status", "Dt_Customer"]
dataframe = label_encode_dataframe(dataframe, categorical_columns)

### DEBATTABLE ACTIONS  
The remaining clean to do is about : outliers & missing values.  
In our case, we'll create 3 datasets to 3 different handles : 
- Dropping missing values & dropping outliers
- Filling missing values with median & dropping outliers
- Filling missing values with median (with outliers)
***

In [5]:
outliers_columns = [
    "Year_Birth",
    "Income",
    "MntWines",
    "MntFruits",
    "MntMeatProducts",
    "MntFishProducts",
    "MntSweetProducts",
    "MntGoldProds",
    "NumDealsPurchases",
    "NumWebPurchases",
    "NumCatalogPurchases",
    "NumWebVisitsMonth",
]

In [6]:
# Drop NA & outliers
dopna_outliers_data = dataframe.copy()

dopna_outliers_data.dropna(inplace=True)
dopna_outliers_data = drop_shared_outliers(dopna_outliers_data, outliers_columns, 90)

dopna_outliers_data.to_csv("../data/cleaned/marketing_campaign_dropna_no_outliers.csv", index=False)

Data points considered outliers for the column Year_Birth:


,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
192,1900,0,2,36640.0,1,0,1,99,15,6,...,1,2,5,0,0,0,0,0,1,0
239,1893,0,4,60182.0,0,1,2,23,8,0,...,0,2,4,0,0,0,0,0,0,0
339,1899,4,5,83532.0,0,0,1,36,755,144,...,6,4,1,0,0,1,0,0,0,0


Data points considered outliers for the column Income:


,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
2233,1977,2,5,666666.0,1,0,1,23,9,14,...,1,3,6,0,0,0,0,0,0,0


Data points considered outliers for the column MntWines:


...

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
1806,1966,4,4,7144.0,0,2,1,92,81,4,...,1,1,0,0,0,0,0,0,0,0
1898,1945,4,4,113734.0,0,0,2,9,6,2,...,0,0,1,0,0,0,0,0,0,0
1975,1969,2,3,4428.0,0,1,1,0,16,4,...,0,0,1,0,0,0,0,0,0,0


Data points considered outliers for the column NumCatalogPurchases:


,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
21,1979,2,3,2447.0,1,0,1,42,1,1,...,28,0,1,0,0,0,0,0,0,0
164,1973,4,3,157243.0,0,1,2,98,20,2,...,22,0,0,0,0,0,0,0,0,0
687,1982,4,3,160803.0,0,0,0,21,55,16,...,28,1,0,0,0,0,0,0,0,0
1653,1977,2,5,157146.0,0,0,1,13,1,0,...,28,0,1,0,0,0,0,0,0,0


Data points considered outliers for the column NumWebVisitsMonth:


,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
9,1950,4,5,5648.0,1,1,2,68,28,0,...,0,0,20,1,0,0,0,0,0,0
774,1957,4,5,6835.0,0,1,0,76,107,2,...,0,1,20,0,0,0,0,0,0,0
1042,1991,2,4,8028.0,0,0,0,62,73,18,...,1,0,19,0,0,0,0,0,0,0
1245,1971,2,2,1730.0,0,0,2,65,1,1,...,0,0,20,0,0,0,0,0,0,0
1846,1963,4,3,4023.0,1,1,2,29,5,0,...,0,0,19,0,0,0,0,0,0,0


In [7]:
# Fill NA & drop outliers


fillna_outliers_data = dataframe.copy()

fillna_outliers_data["Income"].fillna(
    fillna_outliers_data["Income"].median(), inplace=True
)
fillna_outliers_data = drop_shared_outliers(fillna_outliers_data, outliers_columns, 90)

fillna_outliers_data.to_csv("../data/cleaned/marketing_campaign_fillna_no_outliers.csv", index=False)

Data points considered outliers for the column Year_Birth:


/tmp/ipykernel_66230/2255311418.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fillna_outliers_data["Income"].fillna(


,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
192,1900,0,2,36640.0,1,0,1,99,15,6,...,1,2,5,0,0,0,0,0,1,0
239,1893,0,4,60182.0,0,1,2,23,8,0,...,0,2,4,0,0,0,0,0,0,0
339,1899,4,5,83532.0,0,0,1,36,755,144,...,6,4,1,0,0,1,0,0,0,0


Data points considered outliers for the column Income:


,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
2233,1977,2,5,666666.0,1,0,1,23,9,14,...,1,3,6,0,0,0,0,0,0,0


...

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
27,1986,2,4,51381.5,1,0,1,19,5,1,...,0,0,1,0,0,0,0,0,0,0
1806,1966,4,4,7144.0,0,2,1,92,81,4,...,1,1,0,0,0,0,0,0,0,0
1898,1945,4,4,113734.0,0,0,2,9,6,2,...,0,0,1,0,0,0,0,0,0,0
1975,1969,2,3,4428.0,0,1,1,0,16,4,...,0,0,1,0,0,0,0,0,0,0


Data points considered outliers for the column NumCatalogPurchases:


,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
21,1979,2,3,2447.0,1,0,1,42,1,1,...,28,0,1,0,0,0,0,0,0,0
164,1973,4,3,157243.0,0,1,2,98,20,2,...,22,0,0,0,0,0,0,0,0,0
687,1982,4,3,160803.0,0,0,0,21,55,16,...,28,1,0,0,0,0,0,0,0,0
1653,1977,2,5,157146.0,0,0,1,13,1,0,...,28,0,1,0,0,0,0,0,0,0


Data points considered outliers for the column NumWebVisitsMonth:


,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
9,1950,4,5,5648.0,1,1,2,68,28,0,...,0,0,20,1,0,0,0,0,0,0
774,1957,4,5,6835.0,0,1,0,76,107,2,...,0,1,20,0,0,0,0,0,0,0
1042,1991,2,4,8028.0,0,0,0,62,73,18,...,1,0,19,0,0,0,0,0,0,0
1245,1971,2,2,1730.0,0,0,2,65,1,1,...,0,0,20,0,0,0,0,0,0,0
1846,1963,4,3,4023.0,1,1,2,29,5,0,...,0,0,19,0,0,0,0,0,0,0


In [8]:
# Fill NA
fillna_data = dataframe.copy()

fillna_data["Income"].fillna(fillna_data["Income"].median(), inplace=True)

fillna_data.to_csv("../data/cleaned/marketing_campaign_fillna.csv", index=False)

/tmp/ipykernel_66230/1236825801.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fillna_data["Income"].fillna(fillna_data["Income"].median(), inplace=True)
